# Projet numérique : câble sous-marin

## Questions théoriques

1) La loi des grands nombres permet, pour un nombre suffisamment important de simulations conditionnelles, d'estimer l'espérance conditionnelle comme la moyenne empirique de simulations conditionnelles.

2) On note U le vecteur des composantes de Z correspondant aux points de discrétisation sans observation, V celui corresppndant aux points de discrétisation avec observation. 
La loi conditionnelle de U sachant $V=v$ est définie par la densité conditionnelle suivante : 
$f_{U|V = v}(u) = \frac{1}{(2\pi)^{n/2}\sqrt{\det(CS_{U})}}\exp(-\frac{1}{2}(u - m_{U|V=v}){}^tCS_{U}^{-1}(u-m_{U|V=v})$.
En effet, le vecteur des profondeurs $Z$ est gaussien à densité et (au réordonnement des indices près) est tel que $Z = (Z_{1},\ldots,Z_{N})$ avec $U = (Z_1 ,\ldots,Z_{N-n+1}$ et $V = (Z_{N-n},\ldots,Z_N)$.
$CS_{U}$ désigne le complément de Schur du bloc $C_U$ de la matrice de covariance $C$ de $Z$ définie par blocs : $\begin{pmatrix} C_U&C_{U,V} \\ C_{V,U}&C_{V} \end{pmatrix}$ où $C_U = Cov(U,U)$, $C_Z = Cov(V,V)$ et $C_{U,V} = Cov(U,V)$.
On a donc $CS_U = C_U - C_{U,V}C_{V}^{-1}C_{V,U}$.
$m_{U|V=v} = m_U + C_{U,V}C_{V}^{-1}(v-m_V)$ est l'espérance conditionnelle de $U$ sachant $V$ ;  et CS_U est la covariance conditionnelle de $U$ sachant $V$.

3) Si $Y = (Y_1,\ldots,Y_p)$ est un vecteur de composantes gaussiennes indépendantes, toutes d'espérance nulle et de variance 1, alors le vecteur $Z=m+RY$ (où $R$ est une matrice $p\times p$ et $m$ est un vecteur de taille $p$) est gaussien d'espérance $m$ et de matrice de covariance $RR^{t}$.

4) Algorithme de simulation conditionnelle :
Calcul de la matrice de covariance du vecteur des profondeurs $Z$, grâce à la fonction $C$.
On extrait de cette matrice les blocs $C_U$, $C_{U,V}$, $C_{V,U}$,$C_V$ (on tient compte des indices des points d'observation).
Calcul de l'espérance conditionnelle $m$ de $U$ sachant $V$, grâce aux blocs extraits à l'étape précédente.
Calcul de la covariance conditionnelle $CS_U$. Comme la matrice de covariance $C$ du vecteur des profondeurs $Z$ est définie positive, $CS_U$ est aussi définie positive, on peut donc écrire sa décomposition de Cholesky $CS_U = RR^{t}$.
Simulation d'un vecteur gaussien $Y$ de $k$ variables indépendantes d'espérance nulle et de variance 1 (où $k$ est le nombre de points de discrétisation sans observation).
Calcul de $U = m + RY$.
On ainsi effectué une simulation conditionnelle de $U$ sachant $V = v$.

## Données

In [1]:
import numpy as np
from math import*
import matplotlib.pyplot as plt

In [3]:
# Discrétisation

A = 0
B = 500
N = 101  #Nombre de points de discrétisation
Delta = (B-A)/(N-1)
discretization_indexes = np.arange(N)
discretization = Delta * discretization_indexes

# Paramètres du modèle

a = 50
sigma_2 = 12
mu = -5

# Données

observation_indexes = [k for k in range(0, N, 20)]
depth = np.array([0, -4, -12.8, -1, -6.5, 0])

# Indices des composants correspondants aux observations et aux composantes non observ&es

unknown_indexes = list(set(discretization_indexes) - set(observation_indexes))

## Questions

### Covariance

In [7]:
def covariance(a, sigma2, h) :
    return sigma2 * np.exp(-np.abs(h)/a)

X = np.linspace(A, B, N)
distance = np.array([[X[i] - X[j] for j in range(N)] for i in range(N)])
print(f"distances : {distance}")

Sigma = covariance(a, sigma_2, distance)
print(f"matrice de covariance : {Sigma}")

distances : [[   0.   -5.  -10. ... -490. -495. -500.]
 [   5.    0.   -5. ... -485. -490. -495.]
 [  10.    5.    0. ... -480. -485. -490.]
 ...
 [ 490.  485.  480. ...    0.   -5.  -10.]
 [ 495.  490.  485. ...    5.    0.   -5.]
 [ 500.  495.  490. ...   10.    5.    0.]]
matrice de covariance : [[1.20000000e+01 1.08580490e+01 9.82476904e+00 ... 6.65419193e-04
  6.02096185e-04 5.44799157e-04]
 [1.08580490e+01 1.20000000e+01 1.08580490e+01 ... 7.35401941e-04
  6.65419193e-04 6.02096185e-04]
 [9.82476904e+00 1.08580490e+01 1.20000000e+01 ... 8.12744838e-04
  7.35401941e-04 6.65419193e-04]
 ...
 [6.65419193e-04 7.35401941e-04 8.12744838e-04 ... 1.20000000e+01
  1.08580490e+01 9.82476904e+00]
 [6.02096185e-04 6.65419193e-04 7.35401941e-04 ... 1.08580490e+01
  1.20000000e+01 1.08580490e+01]
 [5.44799157e-04 6.02096185e-04 6.65419193e-04 ... 9.82476904e+00
  1.08580490e+01 1.20000000e+01]]


### Extraction

In [9]:
cov_obs, cov_inc, cov_inc_obs, cov_obs_inc = [], [], [], []

for i, L in enumerate(Sigma) :
    for j, c in enumerate(L) :
        if i in observation_indexes and j in observation_indexes :
            cov_obs.append(c)
        elif i in unknown_indexes and j in unknown_indexes :
            cov_inc.append(c)
        elif i in unknown_indexes and j in observation_indexes :
            cov_inc_obs.append(c)
        else :
            cov_obs_inc.append(c)

k = len(unknown_indexes)
cov_obs = np.array(cov_obs).reshape(6, 6)
cov_inc = np.array(cov_inc).reshape(k, k)
cov_inc_obs = np.array(cov_inc_obs).reshape(k, 6)
cov_obs_inc = np.array(cov_obs_inc).reshape(6, k)

print(f" matrice de covariance entre les observations : {cov_obs}")
print (f" matrice de covariance entre les inconnues : {cov_inc}")
print(f" matrice de covariance entre les observations et les inconnues : {cov_obs_inc}")
print(f" matrice de covariance entre les inconnues et les observations : {cov_inc_obs}")

 matrice de covariance entre les observations : [[1.20000000e+01 1.62402340e+00 2.19787667e-01 2.97450261e-02
  4.02555153e-03 5.44799157e-04]
 [1.62402340e+00 1.20000000e+01 1.62402340e+00 2.19787667e-01
  2.97450261e-02 4.02555153e-03]
 [2.19787667e-01 1.62402340e+00 1.20000000e+01 1.62402340e+00
  2.19787667e-01 2.97450261e-02]
 [2.97450261e-02 2.19787667e-01 1.62402340e+00 1.20000000e+01
  1.62402340e+00 2.19787667e-01]
 [4.02555153e-03 2.97450261e-02 2.19787667e-01 1.62402340e+00
  1.20000000e+01 1.62402340e+00]
 [5.44799157e-04 4.02555153e-03 2.97450261e-02 2.19787667e-01
  1.62402340e+00 1.20000000e+01]]
 matrice de covariance entre les inconnues : [[1.20000000e+01 1.08580490e+01 9.82476904e+00 ... 8.12744838e-04
  7.35401941e-04 6.65419193e-04]
 [1.08580490e+01 1.20000000e+01 1.08580490e+01 ... 8.98221959e-04
  8.12744838e-04 7.35401941e-04]
 [9.82476904e+00 1.08580490e+01 1.20000000e+01 ... 9.92688787e-04
  8.98221959e-04 8.12744838e-04]
 ...
 [8.12744838e-04 8.98221959e-04 9.